In [2]:
import pandas as pd
import math
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show

output_notebook()

# Define used columns and load data
cols = ['year', 'state', 'candidate', 'party_detailed', 'candidatevotes', 'totalvotes']
elections_data = pd.read_csv('1976-2020-president.csv', usecols=cols, dtype={'year': str})

# Filter Democratic and Republican parties
party = elections_data['party_detailed']
dem_rep_data = elections_data[(party == 'DEMOCRAT') | (party == 'REPUBLICAN')]

states = dem_rep_data.groupby('state')

# Extract state names
state_names = list(states.groups.keys())

plots = []

# Generate plot for each state
for state_name in state_names:
    state_data = states.get_group(state_name)  
    years = state_data['year'].unique()  

    state_fig = figure(title = state_name.capitalize(), x_range=years, tooltips= '$name total votes: @$name' )
    state_fig.xaxis.major_label_orientation = math.pi/4  #Rotation on x-axis labels

    # Get parties and the vote counts
    parties = state_data['party_detailed'].unique()
    party_votes = state_data.groupby('party_detailed')['candidatevotes']
    total_votes = state_data.groupby('party_detailed')['totalvotes']

    # Calculate vote percentage
    data = {party: party_votes.get_group(party) / total_votes.get_group(party) * 100 for party in parties}
    data['year'] = years

    # Create bar chart
    state_fig.vbar_stack(parties, x='year', color=['blue', 'red'], width=0.8, source=data, legend_label=list(parties))

    #Add the plot list
    plots.append(state_fig)

# Create grid dimensions and display grid
num_cols = math.floor(math.sqrt(len(state_names)))
grid = gridplot(plots, ncols=3, width=300, height=300)
show(grid)


Loading BokehJS ...